In [1]:
import src.bsky

bsky_client = src.bsky.init()

text = "\n".join(await src.bsky.get_author_feed_texts(bsky_client, "zkat.tech", 25))

2025-04-02 21:50:54 [info     ] cache                          adjective=hit key=bsky.get-author-feed-text--zkat.tech prefix=bsky.get-author-feed-text- suffix=zkat.tech
cursor: 2025-03-31T23:05:51.421Z
2025-04-02 21:50:54 [info     ] cache                          adjective=hit key=bsky.get-author-feed-text-2025-03-31T23:05:51.421Z-zkat.tech prefix=bsky.get-author-feed-text-2025-03-31T23:05:51.421Z suffix=zkat.tech
cursor: 2025-03-30T20:25:49.755Z
2025-04-02 21:50:54 [info     ] cache                          adjective=hit key=bsky.get-author-feed-text-2025-03-30T20:25:49.755Z-zkat.tech prefix=bsky.get-author-feed-text-2025-03-30T20:25:49.755Z suffix=zkat.tech
cursor: 2025-03-26T00:46:08.42Z
2025-04-02 21:50:54 [info     ] cache                          adjective=hit key=bsky.get-author-feed-text-2025-03-26T00:46:08.42Z-zkat.tech prefix=bsky.get-author-feed-text-2025-03-26T00:46:08.42Z suffix=zkat.tech
cursor: 2025-03-23T06:03:20.358Z
2025-04-02 21:50:54 [info     ] cache              

In [2]:
import nltk
import nltk.corpus

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/kai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import subprocess
import spacy

subprocess.run(["spacy", "download", "en_core_web_md"])
nlp = spacy.load("en_core_web_md")

In [18]:
def remove_substring_entries(keyword_list):
    # Sort by length of keyword (longest first), then by score (higher is better)
    keyword_list.sort(key=lambda x: (-x[0], -len(x[1])))

    filtered_keywords = []
    seen_words = set()

    for score, keyword in keyword_list:
        words = set(keyword.split())  # Split phrase into individual words
        if not any(word in seen_words for word in words):
            filtered_keywords.append((keyword, score))
            seen_words.update(words)  # Add words to seen set

    return filtered_keywords

In [28]:
import yake
import nltk
import nltk.corpus
import spacy
import yaml
import pprint

with open("stopwords.yml", "r") as _file:
    custom_stopwords = yaml.load(_file, yaml.Loader)

yake_kw_extractor = yake.KeywordExtractor(
    lan="en",
    top=25,
    dedupLim=1,
    stopwords=set(nltk.corpus.stopwords.words("english") + list(nlp.Defaults.stop_words) + custom_stopwords)
)
keywords = yake_kw_extractor.extract_keywords(text.lower())
keywords = list(reversed(sorted(keywords, key=lambda x: x[1])))

print(pprint.pprint(keywords))

keywords = remove_substring_entries(keywords)

print(pprint.pprint(keywords))

keywords = keywords[:5]

print(pprint.pprint(keywords))


[(np.float64(0.007046598930693421), 'works'),
 (np.float64(0.00857653925811423), 'working'),
 (np.float64(0.0019044861974847086), 'work'),
 (np.float64(0.011116710600584122), 'trans women'),
 (np.float64(0.009199714526611918), 'trans folks'),
 (np.float64(0.0018313686854111164), 'trans'),
 (np.float64(0.011768474406249162), 'support'),
 (np.float64(0.007475204770805588), 'stop'),
 (np.float64(0.006475510666285818), 'shit'),
 (np.float64(0.005454958007732468), 'read'),
 (np.float64(0.0022323461434517046), 'qud'),
 (np.float64(0.0037484125965543493), 'love'),
 (np.float64(0.0018397245412873522), 'kdl'),
 (np.float64(0.008966890534985648), 'happy'),
 (np.float64(0.006756965348768421), 'github.com'),
 (np.float64(0.005601332796875475), 'games'),
 (np.float64(0.003907906602471261), 'game'),
 (np.float64(0.008566847834624906), 'fucking'),
 (np.float64(0.005695109769212732), 'folks'),
 (np.float64(0.007460883475051705), 'finally'),
 (np.float64(0.000613928276659871), 'caves of qud'),
 (np.flo

In [ ]:
import spacy
import json

with open("emojis.json", "r") as _file:
    emojis = json.loads(_file.read())

emoji_scores = []

i = 1
for keyword in keywords:
    print(f"progress: {i} out of {len(keywords)}")
    _keyword = nlp(keyword[0])
    keyword_scores = []
    
    for emoji in emojis:
        _emoji = nlp(emoji["description"])
        keyword_score = keyword[1]
        emoji_score = _keyword.similarity(_emoji) * keyword_score
        keyword_scores.append([_keyword, emoji_score])

    print(keyword_scores[0])
    print(keyword_scores[-1])
    keyword_scores.sort(key = lambda x: -x[1])
    emoji_scores.append(keyword_scores[0])
    i += 1

emoji_scores.sort(key = lambda x: -x[1])

progress: 1 out of 5
